# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 3-13 rely on your question 1 answer.

In [115]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [116]:
def load_energy():    
    energy = pd.read_excel('assets/Energy Indicators.xls', header = 17)
    energy = energy.drop(['Unnamed: 0' , 'Unnamed: 1'], axis=1)
    energy.drop(energy.tail(38).index,inplace=True)
    energy.tail()
    energy = energy.rename({'Unnamed: 2': 'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}, axis = 1)

    # For all countries which have missing data (e.g. data with "...") make sure this is reflected as np.NaN values.
    energy['Energy Supply'] = energy['Energy Supply'].replace('...', np.NaN)
    energy['Energy Supply per Capita'] = energy['Energy Supply per Capita'].replace('...', np.NaN)
        
    #There are also several countries with numbers and/or parenthesis in their name. 
    #Be sure to remove these, e.g. 'Bolivia (Plurinational State of)' should be 'Bolivia'. 'Switzerland17' should be 'Switzerland'.
    energy['Country'] = energy['Country'].str.strip('1234567890')
    energy['Country'] = energy['Country'].str.split('(').str[0].str.rstrip()

    
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000 
    energy = energy.set_index('Country')
    energy.rename(index = {"Republic of Korea": "South Korea","United States of America": "United States", 
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", 
    "China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace = True)
    #print(energy.loc['Iran'])
    # columns = ('Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable')
    return energy

load_energy()

,Energy Supply,Energy Supply per Capita,% Renewable
Country,,,
Afghanistan,3.210000e+08,10.0,78.669280
Albania,1.020000e+08,35.0,100.000000
Algeria,1.959000e+09,51.0,0.551010
American Samoa,NaN,NaN,0.641026
Andorra,9.000000e+06,121.0,88.695650
Angola,6.420000e+08,27.0,70.909090
Anguilla,2.000000e+06,136.0,0.000000
Antigua and Barbuda,8.000000e+06,84.0,0.000000
Argentina,3.378000e+09,79.0,24.064520


In [117]:
def load_gdp():
    GDP = pd.read_csv('assets/world_bank.csv', header = 4)
    GDP = GDP.rename(columns={'Country Name': 'Country'})
    GDP = GDP.set_index('Country')
    GDP.rename(index={"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran",
                      "Hong Kong SAR, China": "Hong Kong"}, inplace = True)
    GDP = GDP.drop([str(x) for x in range(1960,2006)], axis=1)
    #GDP = GDP.reset_index()
    #GDP.loc['Iran']
    return GDP

In [118]:
def load_scimen():
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    ScimEn = ScimEn.set_index('Country')
    return ScimEn

In [119]:
load_gdp()
#load_scimen()

,Country Code,Indicator Name,Indicator Code,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,
Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10
Arab World,ARB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.763824e+12,1.863481e+12,1.980983e+12,2.011001e+12,2.103825e+12,2.173896e+12,2.318267e+12,2.385580e+12,2.436659e+12,2.509068e+12
United Arab Emirates,ARE,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,2.789489e+11,2.878318e+11,2.970189e+11,2.814464e+11,2.860493e+11,3.009462e+11,3.216922e+11,3.355937e+11,3.509313e+11,3.621003e+11
Argentina,ARG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,3.788140e+11,4.089887e+11,4.215649e+11,4.217758e+11,4.616402e+11,5.003555e+11,5.043671e+11,5.189199e+11,5.212737e+11,NaN
Armenia,ARM,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,8.679746e+09,9.873142e+09,1.055439e+10,9.060944e+09,9.260285e+09,9.695518e+09,1.039360e+10,1.073658e+10,1.111236e+10,1.144574e+10


In [120]:
#Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). 
#Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).

#The index of this DataFrame should be the name of the country, and the columns should be 
#['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index',
#'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
#'2014', '2015'].

#This function should return a DataFrame with 20 columns and 15 entries, 
#and the rows of the DataFrame should be sorted by "Rank".
    


def answer_one():
    energy = load_energy()
    GDP = load_gdp()
    ScimEn = load_scimen()
    sci_ene = pd.merge(ScimEn, energy, on='Country')
    df = pd.merge(sci_ene, GDP, on='Country')
    df = df[df['Rank'] <= 15]
    df = df.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1)
    return df
    
    #raise NotImplementedError()
    
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [121]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [122]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [123]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [124]:
def answer_two():
    # YOUR CODE HERE
    energy = load_energy()
    GDP = load_gdp()
    ScimEn = load_scimen()
    
    return int(np.max([energy.shape[0], GDP.shape[0], ScimEn.shape[0]]) - 15)
    
    #raise NotImplementedError()
answer_two()

249

In [125]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [126]:
def answer_three():
    # YOUR CODE HERE
    df = answer_one()
    df = df.drop(columns=df.columns[:10], axis=1)
    mean_ = np.mean(df,axis=1).tolist()
    df['avgGDP'] = mean_
    df = df.sort_values(by=['avgGDP'],  ascending=False)
    avgGDP = pd.Series(df['avgGDP'], name='avgGDP', index=df.index)
    return avgGDP
    #raise NotImplementedError()
    
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

In [127]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [128]:
def answer_four():
    # YOUR CODE HERE
    #raise NotImplementedError()
    avggdp = answer_three()
    country = avggdp.index[5]
    df = answer_one()
    df = df.drop(columns=df.columns[:10], axis=1)
    df = df[df.index == country]
    print(df)
    diff = df['2015'].values - df['2006'].values
    return diff[0]
    
answer_four()

                        2006          2007          2008          2009  \
Country                                                                  
United Kingdom  2.419631e+12  2.482203e+12  2.470614e+12  2.367048e+12   

                        2010          2011          2012          2013  \
Country                                                                  
United Kingdom  2.403504e+12  2.450911e+12  2.479809e+12  2.533370e+12   

                        2014          2015  
Country                                     
United Kingdom  2.605643e+12  2.666333e+12  


246702696075.3999

In [129]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [130]:
def answer_five():
    # YOUR CODE HERE
    df = answer_one()
    return np.mean(df['Energy Supply per Capita'])
    #raise NotImplementedError()
#answer_five()

In [131]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [132]:
def answer_six():
    df = answer_one()
    max_ = np.max(df['% Renewable'])
    six = df.index[df['% Renewable'] == max_].tolist()
    six.append(max_)
    
    return tuple(six)
    
    #raise NotImplementedError()
answer_six()

('Brazil', 69.64803)

In [133]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [134]:
def answer_seven():
    # YOUR CODE HERE
    df = answer_one()
    df['Citations Ratio'] = (df['Self-citations'])/(df['Citations'])
    max_ = np.max(df['Citations Ratio'])
    seven = df.index[df['Citations Ratio'] == max_].tolist()
    seven.append(max_)
    
    return tuple(seven)
    #raise NotImplementedError()
answer_seven()

('China', 0.6893126179389422)

In [135]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [136]:
def answer_eight():
    df = answer_one()
    df['PopEst'] = (df['Energy Supply'])/(df['Energy Supply per Capita'])
    df = df.sort_values(by=['PopEst'], ascending=False)
    return df.iloc[[2]].index.tolist()[0]
    #raise NotImplementedError()
answer_eight()

'United States'

In [137]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [138]:
def answer_nine():
    # YOUR CODE HERE
    df = answer_one()
    df['PopEst'] = df['Energy Supply'] / df['Energy Supply per Capita']
    df['Citable docs per Capita'] = df['Citable documents'] / df['PopEst']
    matrix = df.corr()
    #plot9()
    return matrix.iloc[[21]]['Energy Supply per Capita'].tolist()[0]
    #raise NotImplementedError()
    
answer_nine()

0.7940010435442943

In [139]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [140]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [219]:
def answer_ten():
    df = answer_one()
    median = np.median(df['% Renewable'])
    print(median)
    df['Above Median'] = [1 if x >= median else 0 for x in df['% Renewable']]
    df.sort_values(by='Rank', ascending = True, inplace=True)
    print(df.index.tolist())
    return pd.Series(df['Above Median'], name='HighRenew', index=df.index)
answer_ten()

17.02028
['China', 'United States', 'Japan', 'United Kingdom', 'Russian Federation', 'Canada', 'Germany', 'India', 'France', 'South Korea', 'Italy', 'Spain', 'Iran', 'Australia', 'Brazil']


Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

In [220]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


17.02028
['China', 'United States', 'Japan', 'United Kingdom', 'Russian Federation', 'Canada', 'Germany', 'India', 'France', 'South Korea', 'Italy', 'Spain', 'Iran', 'Australia', 'Brazil']


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [194]:
def answer_eleven():
    df = answer_one()
    df['PopEst'] = df['Energy Supply'] / df['Energy Supply per Capita']
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    df['Continent'] = [ContinentDict[k] for k in df.index.tolist()]
    df = df.reset_index()
    df1 = pd.DataFrame(df,columns=('Country', 'PopEst', 'Continent'))
    df1 = df1.groupby('Continent').agg({'PopEst': ['count','sum','mean','std']})
    df1.columns = df1.columns.droplevel()
    df1.rename(columns={'count':'size'}, inplace = True)
    return df1
    #raise NotImplementedError()
answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [195]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [216]:
def answer_twelve():
    # YOUR CODE HERE
    df = answer_one()
    df['PopEst'] = df['Energy Supply'] / df['Energy Supply per Capita']
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    df['Continent'] = [ContinentDict[k] for k in df.index.tolist()]
    df = df.reset_index()
    df1 = pd.DataFrame(df,columns=('% Renewable','Country', 'Continent'))
    df1['Bins'] = pd.qcut(df['% Renewable'], q=5, precision = 0)
    df1 = df1.groupby(['Continent', 'Bins']).agg({'Country': ['count']})
    df1.columns = df1.columns.droplevel()
    return pd.Series(df1['count'], index=df1.index)
    #return pd.qcut(df1['% Renewable'], q=5)
    #raise NotImplementedError()
answer_twelve()

Continent      Bins        
Asia           (1.0, 11.0]     3
               (14.0, 18.0]    1
               (18.0, 35.0]    1
Australia      (11.0, 14.0]    1
Europe         (11.0, 14.0]    1
               (14.0, 18.0]    2
               (18.0, 35.0]    2
               (35.0, 70.0]    1
North America  (11.0, 14.0]    1
               (35.0, 70.0]    1
South America  (35.0, 70.0]    1
Name: count, dtype: int64

In [217]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


AssertionError: Q12: Wrong result numbers!

### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [190]:
def answer_thirteen():
    # YOUR CODE HERE
    df = answer_one()
    df['PopEst'] = df['Energy Supply'] / df['Energy Supply per Capita']
    df['PopEst'] = pd.to_numeric(df['PopEst'])
    df['PopEst'] =  df['PopEst'].apply('{:,}'.format)
    return pd.Series(df['PopEst'], name='PopEst', index=df.index)
    #raise NotImplementedError()
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

In [191]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")